In [ ]:
import numba
import numpy as np
from math import sqrt

%load_ext Cython

In [ ]:
@numba.jit('float64 (float64[:], float64[:])')
def metric_numba(x, y):
    """
    standard Euclidean distance
    """
    ret = x-y
    ret *= ret
    return np.sqrt(ret).sum()


@numba.jit('float64 (float64[:], float64[:,:])', nopython=True)
def inf_dist_numba(x, Y):
    """
    inf distance between row x and array Y
    """
    m = Y.shape[0]
    inf = np.inf
    
    for i in range(m):
        dist = metric_numba(x, Y[i])
        if dist < inf:
            inf = dist
    return inf

@numba.jit('float64 (float64[:,:], float64[:,:])', nopython=True)
def hausdorff_numba(X, Y):
    """
    Hausdorff distance between arrays X and Y
    """
    m = X.shape[0]
    n = Y.shape[0]
    sup1 = -1.
    sup2 = -1.
    
    for i in range(m):
        inf1 = inf_dist_numba(X[i], Y)
        if inf1 > sup1:
            sup1 = inf1
    for i in range(n):
        inf2 = inf_dist_numba(Y[i], X)
        if inf2 > sup2:
            sup2 = inf2
            
    return max(sup1, sup2)

In [ ]:
%%cython -a -c=-fPIC -c=-fwrapv -c=-O3 -c=-fno-strict-aliasing
#!python
#cython: cdivision=True, boundscheck=False, nonecheck=False, wraparound=False, initializedcheck=False

import numpy as np
cimport numpy as cnp
ctypedef cnp.float64_t float64_t
from libc.math cimport sqrt

cdef float64_t metric_cython(float64_t[::1] x,float64_t[::1] y):
    cdef:
        int i = 0
        int n = x.shape[0]
        float ret = 0
    for i in range(n):
        ret += (x[i]-y[i])**2
    return sqrt(ret)

cdef float64_t inf_dist_cython(float64_t[::1] x, float64_t[:,::1] Y):
    cdef:
        int m = Y.shape[0]
        float inf = cnp.inf
        float dist
    
    for i in range(m):
        dist = metric_cython(x, Y[i,:])
        if dist < inf:
            inf = dist
    return inf

def hausdorff_cython(float64_t[:,::1] X, float64_t[:,::1] Y):
    cdef:
        int m = X.shape[0]
        int n = Y.shape[0]
        float sup1 = -1.
        float sup2 = -1.
    
    for i in range(m):
        inf1 = inf_dist_cython(X[i,:], Y)
        if inf1 > sup1:
            sup1 = inf1
    for i in range(n):
        inf2 = inf_dist_cython(Y[i,:], X)
        if inf2 > sup2:
            sup2 = inf2
            
    return max(sup1, sup2)

In [ ]:
for i in np.arange(1,11):
    n = i*100
    m = 100
    X = np.random.random((n,m))
    Y = np.random.random((n,m))
    print("Iteración " + str(i) +" con Numba:")
    %timeit hausdorff_numba(X,Y)

In [ ]:
for i in np.arange(1,11):
    n = i*100
    m = 100
    X = np.random.random((n,m))
    Y = np.random.random((n,m))
    print("Iteración " + str(i) +" con Cython:")
    %timeit hausdorff_cython(X,Y)